In [1]:
!pip install faker


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# Inicializar Faker y la semilla para reproducibilidad
fake = Faker()
Faker.seed(0)
random.seed(0)

# Definir el número de filas
n = 10000  # Ajustable según el tamaño deseado del dataset

# Generar un conjunto único de IDs de usuario
unique_user_ids = random.sample(range(1, 13001), 13000)

# Definir periodos y actividades
periodos = ["2020-1", "2020-2", "2021-1", "2021-2", "2022-1", "2022-2", "2023-1", "2023-2", "2024-1"]
actividades_periodo = [
    "Inicio de Ciclo", "Primera Práctica Calificada", "Segunda Práctica Calificada", 
    "Examen Parcial", "Tercera Práctica Calificada", "Cuarta Práctica Calificada", 
    "Examen Final", "Examen Sustitutorio", "Cierre de Ciclo"
]
# Asignar fechas de inicio y fin para cada periodo
periodo_fechas = {
    "2020-1": ("2020-01-01", "2020-06-30"),
    "2020-2": ("2020-07-01", "2020-12-31"),
    "2021-1": ("2021-01-01", "2021-06-30"),
    "2021-2": ("2021-07-01", "2021-12-31"),
    "2022-1": ("2022-01-01", "2022-06-30"),
    "2022-2": ("2022-07-01", "2022-12-31"),
    "2023-1": ("2023-01-01", "2023-06-30"),
    "2023-2": ("2023-07-01", "2023-12-31"),
    "2024-1": ("2024-01-01", "2024-06-30"),
}

roles = ["Docente", "Estudiante"]
tipo_notificacion = ["Recordatorio", "Preparación", "Asistencia", "Justificación", "Seguimiento"]
estado_notificacion = ["Enviada", "Pendiente", "Fallida"]
exito_entrega = ["Éxito", "Fallo"]
efectividad = ["Alta", "Media", "Baja", "Nula"]

# Función para generar una fecha aleatoria dentro de un rango específico
def generar_fecha_aleatoria(fecha_inicio, fecha_fin):
    start_date = datetime.strptime(fecha_inicio, "%Y-%m-%d")
    end_date = datetime.strptime(fecha_fin, "%Y-%m-%d")
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

# Generar datos
data = []
for _ in range(n):
    # Seleccionar un usuario_id y rol
    usuario_id = random.choice(unique_user_ids)
    rol = random.choice(roles)
    
    # Seleccionar periodo y actividad
    periodo = random.choice(periodos)
    actividad = random.choice(actividades_periodo)
    
    # Fecha de notificación según el periodo
    fecha_inicio, fecha_fin = periodo_fechas[periodo]
    fecha_notificacion = generar_fecha_aleatoria(fecha_inicio, fecha_fin)
    
    # Configuración de tiempos de notificación para maximizar respuesta
    if actividad in ["Examen Parcial", "Examen Final"]:
        dias_antes = random.choice([3, 5, 7])  # Exámenes con mayor anticipación
    elif actividad in ["Primera Práctica Calificada", "Segunda Práctica Calificada", "Tercera Práctica Calificada", "Cuarta Práctica Calificada"]:
        dias_antes = random.choice([1, 3, 5])  # Prácticas calificadas con menos anticipación
    else:
        dias_antes = random.choice([0, 1, 2])  # Actividades generales
    
    minutos_antes = random.choice([0, 10, 15]) if actividad in ["Inicio de Ciclo", "Cierre de Ciclo"] else 0
    horas_despues = random.choice([0, 1, 2]) if actividad in ["Inicio de Ciclo", "Examen Final"] and dias_antes == 0 else 0

    # Estado de notificación y éxito
    estado = random.choices(estado_notificacion, weights=[0.7, 0.2, 0.1], k=1)[0]
    exito = "Éxito" if estado == "Enviada" else "Fallo"
    
    # Probabilidad de respuesta y efectividad en base al tipo de notificación
    respuesta_recibida = 1 if exito == "Éxito" and random.random() > 0.3 else 0
    tiempo_respuesta = random.randint(1, 30) if respuesta_recibida == 1 else 0
    efectividad_nota = random.choice(["Alta", "Media", "Baja"]) if respuesta_recibida == 1 else "Nula"
    
    # Añadir fila de datos
    data.append({
        "Periodo": periodo,
        "FechaNotificacion": fecha_notificacion.strftime("%Y-%m-%d"),
        "UsuarioID": usuario_id,
        "Rol": rol,
        "Actividad": actividad,
        "TipoNotificacion": random.choice(tipo_notificacion),
        "DiasAntes": dias_antes,
        "MinutosAntes": minutos_antes,
        "HorasDespues": horas_despues,
        "EstadoNotificacion": estado,
        "RespuestaRecibida": respuesta_recibida,
        "TiempoRespuesta": tiempo_respuesta,
        "ExitoEntrega": exito,
        "Efectividad": efectividad_nota
    })

# Convertir a DataFrame
df = pd.DataFrame(data)

# Guardar en archivo CSV
df.to_csv("notificaciones_academicas.csv", index=False)

# Mostrar las primeras filas para verificar
print(df.head())


  Periodo FechaNotificacion  UsuarioID         Rol  \
0  2022-2        2022-11-03       7605     Docente   
1  2022-2        2022-08-11       9112  Estudiante   
2  2021-1        2021-02-14       8490  Estudiante   
3  2020-1        2020-02-21       1231     Docente   
4  2023-2        2023-10-17       4417     Docente   

                     Actividad TipoNotificacion  DiasAntes  MinutosAntes  \
0              Inicio de Ciclo    Justificación          2            10   
1  Tercera Práctica Calificada      Preparación          5             0   
2          Examen Sustitutorio    Justificación          0             0   
3  Segunda Práctica Calificada    Justificación          1             0   
4               Examen Parcial      Preparación          5             0   

   HorasDespues EstadoNotificacion  RespuestaRecibida  TiempoRespuesta  \
0             0            Enviada                  1               17   
1             0            Enviada                  1               27